# ASTI-ALaM Implementation for DTW-Based Crop Mapping
## Using DTAIDistance module from KU Leuven
https://dtaidistance.readthedocs.io/en/latest/usage/dtw.html

## Version 1.0β1

### Notable Changes
- Transformed sequential mother code contents into functions

# Initializations

## Modules

In [ ]:
# Core geostatistical modules
import pandas as pd
import geopandas as gpd
import numpy as np

# Formatting modules
import os, glob
import datetime as dt
import re # module for extracting and tabulating date information from filename
from tqdm import tqdm # module for visual progress bars
import warnings # module for suppressing currently annoying Pandas update warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data preparation and processing modules
import math # module for quick detection of NaN pixels
import rioxarray as rxr # module for xarray-based recording of GeoTIFF data
import xarray as xr
from geocube.api.core import make_geocube # module for conversion of vectors into rasters
from dtaidistance import dtw # the mother module for obtaining the distance accumulated cost
from dtaidistance import dtw_ndim # accessory module for testing multi-dimensional DTW

# Data plotting and visualization modules
from rasterio.plot import plotting_extent # module for obtaining bounding boxes based on trimmed image
import earthpy.plot as ep # module for fast plotting of GeoTIFF images
import matplotlib.pyplot as plt
import altair as alt # module for plot charting
from dtaidistance import dtw_visualisation as dtwvis # accessory module for visualizing DTW results
from IPython.display import Image # module for displaying image outputs in Jupyter

%matplotlib inline

## Sentinel-1 parameter configurations

In [ ]:
SCAN_MODE = "dsc" # "asc" for ascending orbit, "dsc" for descending orbit
BAND = 2 # 1 = VV, 2 = VH, 3 = angle

## DTW parameter configurations

In [ ]:
DTW_WINDOW_SIZE = 5 # allowance for dynamic diagonal shifting, using 1 will make the algorithm calculate the Euclidean distance
DTW_PSI = 2 # PSI (Prefix and Suffix-Invariant) relaxation parameter
DTW_MAX_DIST = float("inf") # avoid computing partial paths that will be larger than this value, returning infinity
DTW_USE_PRUNING = True # automates determination of the above MAX_DIST parameter
DTW_USE_C = True # DTAIdistance-exclusive call to use pure C-based compiled functions (default is False)
# NOTE: to use DTW_USE_C, the array inputs must be of type np.double

DTW_COST_THRESHOLD = 20 # maximum allowable distance cost threshold before pixels are masked out of the final map

## Data directory configurations

In [ ]:
CROP_REFERENCE_GEOMETRY_DIR = "shp/" # Set directory for features program will use to generate a reference temporal signature
STUDY_AREA_GEOMETRY_DIR = "shp/" # Set directory for the boundaries of your study area
CROP_REFERENCE_CATALOG = "img_data/" # Set directory for the catalog of time-series Sentinel-1 images to extract the reference temporal signature
CROP_TEST_CATALOG = "img_data/" # Set directory for the catalog of time-series Sentinel-1 images to compare with the reference temporal signature

DATE_FORMAT = "%Y-%m-%d" # Date format to use to derive date information from image filenames

In [ ]:
# Shapefile for for generating reference temporal signature
CROP_REFERENCE_TYPE = "Your crop" # to be appended in the output filename
CROP_REFERENCE_GEOMETRY = gpd.read_file(CROP_REFERENCE_GEOMETRY_DIR)

# Study area boundary shapefile
STUDY_AREA_NAME = "Your study area" # to be appended in the output filename
STUDY_AREA_GEOM = gpd.read_file(STUDY_AREA_GEOMETRY_DIR)

# Preprocessed Sentinel-1 reference and test data paths
CROP_REFERENCE_YEAR = '2018' # to be appended in the output filename as r(year)
CROP_REFERENCE_FOLDER_PATH = CROP_REFERENCE_CATALOG + "_" + CROP_REFERENCE_YEAR
START_OF_CROP_REFERENCE_YEAR = CROP_REFERENCE_YEAR + '-01-01'

TEST_YEAR = '2017' # to be appended in the output filename as t(year)
TEST_FOLDER_PATH = CROP_TEST_CATALOG + "_" + TEST_YEAR
START_OF_TEST_YEAR = TEST_YEAR + '-01-01'

## Function definitions

### Temporal signature extraction

In [ ]:
def generate_temporal_signature(band,
                                scan_mode,
                                start_of_crop_reference_year,
                                date_format,
                                crop_reference_folder_path,
                                crop_reference_geometry):
    
    crop_reference_list = []
    for filename in glob.glob(os.path.join(crop_reference_folder_path, '*' + scan_mode + '*.tif')):
        with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
            date = re.search("([0-9]{4}[0-9]{2}[0-9]{2})", filename).group()
            formatted_date = date[:4] + "-" + date[4:6] + "-" + date[6:8]
            crop_reference_list.append([formatted_date, rxr.open_rasterio(filename,
                                masked=True
                                ).rio.clip(
                                crop_reference_geometry.geometry.values, crop_reference_geometry.crs, from_disk=True
                                )])
    # Call for results checking
    titles = [band]
    ep.plot_bands(crop_reference_list[0][1].sel(band=band), title=titles)
    
    crop_reference_temporal_signature = []
    for x in crop_reference_list:
    # Temporary array for converting zeroes to NaN for obtaining actual mean of existing pixels
        temp = x[1].sel(band=BAND).data
        temp[temp == 0] = 'nan'
        crop_reference_temporal_signature.append([(
            dt.datetime.strptime(x[0], date_format) - dt.datetime.strptime(start_of_crop_reference_year, date_format)).days,
            np.nanmean(temp)])

    crop_reference_temporal_signature = np.array(crop_reference_temporal_signature)
    crop_reference_temporal_signature[crop_reference_temporal_signature[:, 0].argsort()]
    crop_reference_temporal_signature[np.isnan(crop_reference_temporal_signature)] = 0
    crop_reference_temporal_signature_df = pd.DataFrame(
        crop_reference_temporal_signature).rename(columns = {0: 'days', 1: 'band ' + str(band)})

    crop_reference_temporal_signature_1d = np.array([row[1] for row in crop_reference_temporal_signature]).astype(np.double)
    return crop_reference_temporal_signature_1d, crop_reference_temporal_signature_df

### Prepare the test dataset

In [ ]:
def generate_test_dataset(band,
                          scan_mode,
                          test_folder_path,
                          study_area_geom):
    
    test_data_list = []

    for filename in glob.glob(os.path.join(test_folder_path, '*' + scan_mode + '*.tif')):
        with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
            date = re.search("([0-9]{4}[0-9]{2}[0-9]{2})", filename).group()
            formatted_date = date[:4] + "-" + date[4:6] + "-" + date[6:8]
            test_data_list.append([formatted_date, rxr.open_rasterio(filename,
                                masked=True
                                ).rio.clip(
                                study_area_geom.geometry.values, study_area_geom.crs, from_disk=True
                                )])
            
    # Making a sample plot of the test dataset
    titles = [band]
    ep.plot_bands(test_data_list[0][1].sel(band=band), title=titles)
    
    # Creation of new accessory constants based on test dataset
    img_dimensions = test_data_list[0][1].sel(band=band).shape
    
    # Formulation of equivalents to pixel dimensions in lat-long degree measure
    lon_increment = (study_area_geom.bounds['maxx'][0] - study_area_geom.bounds['minx'][0]) / img_dimensions[1]
    lat_increment = (study_area_geom.bounds['maxy'][0] - study_area_geom.bounds['miny'][0]) / img_dimensions[0]

    # CAUTION: Keep in mind that an image's Cartesian y-axis may be the opposite of a geographical y-axis
    print("Longitude Increment: ", lon_increment, "\nLatitude Decrement: ", lat_increment)
    
    return test_data_list, img_dimensions, lon_increment, lat_increment

### DTW-based accumulated distance cost in map form
### The heart (is the strongest muscle) of the code

In [ ]:
def generate_accumulated_distance_cost_map(test_data_list,
                                           band,
                                           study_area_geom,
                                           start_of_test_year,
                                           date_format,
                                           crop_parcel_reference_temporal_signature_1d,
                                           dtw_window_size,
                                           dtw_psi,
                                           dtw_max_dist,
                                           dtw_use_pruning,
                                           dtw_use_c,
                                           img_dimensions,
                                           lon_increment,
                                           lat_increment):
    
    pixel_cost_array_df = pd.DataFrame(columns = ["distance_cost", "x", "longitude", "y", "latitude"])

    lon_track = study_area_geom.bounds["minx"][0]

    for col in tqdm(range(img_dimensions[1] - 1)):
        lat_track = study_area_geom.bounds["maxy"][0]
    
        for row in range(img_dimensions[0] - 1):
            pixel_timeseries_array = [] # Temporarily stores date and pixel data array information for DTW computation
        
            for count in range(len(test_data_list)):
                # Skips blank and NaN pixels
                if (test_data_list[count][1].sel(band=band).data[row][col] == 0. or
                    math.isnan(test_data_list[count][1].sel(band=band).data[row][col])):
                    continue
            
                # Procedures for valid pixels
                else:
                    day_of_acquisition = (dt.datetime.strptime(test_data_list[count][0], date_format) -
                        dt.datetime.strptime(start_of_test_year, date_format)).days
                    pixel_value = test_data_list[count][1].sel(band=band).data[row][col]
                    pixel_timeseries_array.append([day_of_acquisition, pixel_value])
                
            # Skips the DTW calculator phase if timeseries array is empty
            if not pixel_timeseries_array:
                lat_track = lat_track - lat_increment
                continue
            
            # activates the DTW calculator phase for non-empty array
            else:
                #pixel_timeseries_array[pixel_timeseries_array[:, 0].argsort()]
                pixel_timeseries_array_1d = np.array([row[1] for row in pixel_timeseries_array]).astype(np.double)
                distance_1d = dtw.distance(
                    pixel_timeseries_array_1d,
                    crop_parcel_reference_temporal_signature_1d,
                    window=dtw_window_size,
                    psi=dtw_psi,
                    #max_dist = dtw_max_dist,
                    use_pruning = dtw_use_pruning,
                    use_c = dtw_use_c
                )
                pixel_cost_array_df.loc[len(pixel_cost_array_df)] = [distance_1d, col, lon_track, row, lat_track]
                lat_track = lat_track - lat_increment
            
        lon_track = lon_track + lon_increment
    
    return pixel_cost_array_df

# The Main Body

## Creating the reference temporal signature based on scan mode

In [ ]:
crop_reference_temporal_signature_1D, crop_reference_temporal_signature_df = \
    generate_temporal_signature(BAND,
                                SCAN_MODE,
                                START_OF_CROP_REFERENCE_YEAR,
                                DATE_FORMAT,
                                CROP_REFERENCE_FOLDER_PATH,
                                CROP_REFERENCE_GEOMETRY)

alt.Chart(crop_reference_temporal_signature_df).mark_line().encode(
    x = 'days',
    y = 'band ' + str(BAND)
)

## Preparing the test dataset

In [ ]:
test_data_list, IMG_DIMENSIONS, LON_INCREMENT, LAT_INCREMENT = generate_test_dataset(BAND,
                                                                                     SCAN_MODE,
                                                                                     TEST_FOLDER_PATH,
                                                                                     STUDY_AREA_GEOM)

## Generation of accumulated distance cost map

In [ ]:
accumulated_distance_cost_map_df = generate_accumulated_distance_cost_map(test_data_list,
                                                                          BAND,
                                                                          STUDY_AREA_GEOM,
                                                                          START_OF_TEST_YEAR,
                                                                          DATE_FORMAT,
                                                                          crop_reference_temporal_signature_1D,
                                                                          DTW_WINDOW_SIZE,
                                                                          DTW_PSI,
                                                                          DTW_MAX_DIST,
                                                                          DTW_USE_PRUNING,
                                                                          DTW_USE_C,
                                                                          IMG_DIMENSIONS,
                                                                          LON_INCREMENT,
                                                                          LAT_INCREMENT)

In [ ]:
accumulated_distance_cost_map_df

# Output Generation

## Generates the full map

In [ ]:
accumulated_distance_cost_map_gdf = gpd.GeoDataFrame(
    accumulated_distance_cost_map_df, geometry=gpd.points_from_xy(
        accumulated_distance_cost_map_df.longitude, accumulated_distance_cost_map_df.latitude), crs="EPSG:4326"
)

# Conversion to raster form
out_grid = make_geocube(
    vector_data=accumulated_distance_cost_map_gdf,
    measurements=['distance_cost'],
    resolution=(-LON_INCREMENT, LAT_INCREMENT),
)

output_filename = "out/distmap_" + STUDY_AREA_NAME + "_" + CROP_REFERENCE_TYPE + "_" + SCAN_MODE +  "_b" + str(BAND) + \
    "_refpclyear"+ str(CROP_REFERENCE_YEAR) + "_testyear"+ str(TEST_YEAR) + "_dtw_ws" + str(DTW_WINDOW_SIZE) + \
    "_psi" + str(DTW_PSI) + "_pr" + str(DTW_USE_PRUNING) + "_C" + str(DTW_USE_C) + "_full.tif"
out_grid.rio.to_raster(output_filename)

## Generates the map with cost threshold

In [ ]:
MAP_COST_THRESHOLD = 20

accumulated_distance_cost_map_gdf_thr = accumulated_distance_cost_map_gdf[
    accumulated_distance_cost_map_gdf.distance_cost <= MAP_COST_THRESHOLD]

# Conversion to raster form
out_grid = make_geocube(
    vector_data=accumulated_distance_cost_map_gdf_thr,
    measurements=['distance_cost'],
    resolution=(-LON_INCREMENT, LAT_INCREMENT),
)

output_filename = "out/distmap_" + STUDY_AREA_NAME + "_" + CROP_REFERENCE_TYPE + "_" + SCAN_MODE +  "_b" + str(BAND) + \
    "_refpclyear"+ str(CROP_REFERENCE_YEAR) + "_testyear"+ str(TEST_YEAR) + "_dtw_ws" + str(DTW_WINDOW_SIZE) + \
    "_psi" + str(DTW_PSI) + "_pr" + str(DTW_USE_PRUNING) + "_C" + str(DTW_USE_C) + "_t" + str(MAP_COST_THRESHOLD) + ".tif"
out_grid.rio.to_raster(output_filename)